<a href="https://colab.research.google.com/github/kb10241024/PYNBs/blob/master/VGGNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[BLOG](https://medium.com/datadriveninvestor/five-powerful-cnn-architectures-b939c9ddd57b)

# VGGNet — Simonyan et al
The runner up of 2014 imagenet challenge is named VGGNet. Because of the simplicity of it’s uniform architecture, it appeals to a new-comer as a simpler form of a deep convolutional neural network.


In future posts, we will see how this network is one of the most used choices for feature extraction from images (taking images and converting them to a smaller dimensional array that contains important information regarding the image).

![alt text](https://miro.medium.com/max/919/1*bLzGoL9g9Sj43xkTbiyAzw.png)

# VGGNet — Architecture
VGGNet has 2 simple rules of thumb to be followed:
Each Convolutional layer has configuration — kernel size = 3×3, stride = 1×1, padding = same. The only thing that differs is number of filters.

Each Max Pooling layer has configuration — windows size = 2×2 and stride = 2×2. Thus, we half the size of the image at every Pooling layer.

The input image was an RGB image of 224×224 pixels. So input size = 224x224x3

![alt text](https://miro.medium.com/max/1128/1*2C_PcSh4t5IjppjN7v_Gmw.png)

Total Params = 138 million. Most of these parameters are contributed by fully connected layers.

*   The first FC layer contributes = 4096 * (7 * 7 * 512) + 4096 = 102,764,544
*   The second FC layer contributes = 4096 * 4096 + 4096 = 16,781,312
*   The third FC layer contributes = 4096 * 1000 + 4096 = 4,100,096
Total params contributed by FC layers = 123,645,952.



# VGGNet — CODE

In [1]:
from keras import layers
from keras.models import Model, Sequential

from functools import partial

conv3 = partial(layers.Conv2D,
                kernel_size=3,
                strides=1,
                padding='same',
                activation='relu')

def block(in_tensor, filters, n_convs):
    conv_block = in_tensor
    for _ in range(n_convs):
        conv_block = conv3(filters=filters)(conv_block)
    return conv_block

def _vgg(in_shape=(227,227,3),
         n_classes=1000,
         opt='sgd',
         n_stages_per_blocks=[2, 2, 3, 3, 3]):
    in_layer = layers.Input(in_shape)

    block1 = block(in_layer, 64, n_stages_per_blocks[0])
    pool1 = layers.MaxPool2D()(block1)
    block2 = block(pool1, 128, n_stages_per_blocks[1])
    pool2 = layers.MaxPool2D()(block2)
    block3 = block(pool2, 256, n_stages_per_blocks[2])
    pool3 = layers.MaxPool2D()(block3)
    block4 = block(pool3, 512, n_stages_per_blocks[3])
    pool4 = layers.MaxPool2D()(block4)
    block5 = block(pool4, 512, n_stages_per_blocks[4])
    pool5 = layers.MaxPool2D()(block5)
    flattened = layers.GlobalAvgPool2D()(pool5)

    dense1 = layers.Dense(4096, activation='relu')(flattened)
    dense2 = layers.Dense(4096, activation='relu')(dense1)
    preds = layers.Dense(1000, activation='softmax')(dense2)

    model = Model(in_layer, preds)
    model.compile(loss="categorical_crossentropy", optimizer=opt,
	              metrics=["accuracy"])
    return model

def vgg16(in_shape=(227,227,3), n_classes=1000, opt='sgd'):
    return _vgg(in_shape, n_classes, opt)

def vgg19(in_shape=(227,227,3), n_classes=1000, opt='sgd'):
    return _vgg(in_shape, n_classes, opt, [2, 2, 4, 4, 4])

if __name__ == '__main__':
    model = vgg19()
    print(model.summary())

Using TensorFlow backend.








Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 227, 227, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 227, 227, 64)      1792      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 227, 227, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 113, 113, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 113, 113, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 113, 113, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 56, 56, 128)     